In [1]:
!pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
import pandas as pd
df = pd.read_excel(r'C:\Users\Admin\Desktop\OMJ_PO.xlsx', skiprows=10) ## Read the Excel file starting from row 11 (skip first 10 rows)
df = df.dropna(subset=['OMJ Style # ']).copy()

df.head(10)

,OMJ Style #,Elegant Jewelry Style #,Quantity,Metal Type,Metal Color,SIZE,Shank Thickness,Shank Width,PO #,Diamod Quality,Finish Type #,Delivery Time
0,VAL2512-75,YR4171G,1.0,14K,YELLOW,9.50,1.51,2.80,ALGUL,W5A / G+- SI1,COMPLETE BAND,7 Days
1,VAL2512-75,YR4171G,1.0,14K,WHITE,8.25,1.51,2.80,001-02398,W5A / G+- SI1,COMPLETE BAND,7 Days
2,VAL1915-50,YR4171F,1.0,14k,YELLOW,7.00,1.36,2.44,001-00572,W5A / G+- SI1,COMPLETE BAND,7 Days
4,VAL1915-50,YR4171F,1.0,14K,YELLOW,5.00,1.36,2.44,001-114604,W5A / G+- SI1,COMPLETE BAND,7 Days
6,VAL3310-140,YR4171SB,2.0,14K,WHITE,9.00,1.70,3.54,ATHMANN,NaN,CASTING ONLY,7 Days
7,VAL3110-125,YR4171S,1.0,14K,WHITE,6.00,1.63,3.41,ZAVAL,NaN,CASTING ONLY,7 Days
9,ZOE1321-20,YR4321CMC,1.0,14K,WHITE,4.25,1.34,1.94,001-114604,W5A / G+- SI1,COMPLETE BAND,7 Days
10,BND999-25,RG0000361D,2.0,14K,WHITE,6.00,1.26,1.72,661070S,W5A / G+- SI1,COMPLETE BAND,7 Days
14,PLEASE LASER OM LOGO - METAL TYPE - RING ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,PLEASE MAKE SURE THERE IS NO POROSITY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
import pandas as pd
import re

# Step 1: Read the actual data starting from row 11 (skip first 10 rows)
df = pd.read_excel(r'C:\Users\Admin\Desktop\OMJ_PO.xlsx', skiprows=10) 

# Step 2: Select required columns
selected_columns = ['OMJ Style # ', 'Elegant Jewelry Style #', 'Quantity ', 
                    'Metal Type', 'Metal Color ', 'SIZE', 
                    'Shank Thickness', 'Shank Width', 'PO # ', 'Diamod Quality ']
df_selected = df[selected_columns]

# Step 3: Drop rows where 'OMJ Style # ' is missing
df_cleaned = df_selected.dropna(subset=['OMJ Style # ']).copy()

# Step 4: Convert numeric columns safely
# Quantity
df_cleaned['Quantity '] = pd.to_numeric(df_cleaned['Quantity '], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Quantity '].notna()]
df_cleaned['Quantity '] = df_cleaned['Quantity '].astype(int)

# SIZE
df_cleaned['SIZE'] = pd.to_numeric(df_cleaned['SIZE'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['SIZE'].notna()]
df_cleaned['SIZE'] = df_cleaned['SIZE'].astype(float)

# Shank Thickness
df_cleaned['Shank Thickness'] = pd.to_numeric(df_cleaned['Shank Thickness'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Thickness'].notna()]
df_cleaned['Shank Thickness'] = df_cleaned['Shank Thickness'].astype(float)

# Shank Width
df_cleaned['Shank Width'] = pd.to_numeric(df_cleaned['Shank Width'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Width'].notna()]
df_cleaned['Shank Width'] = df_cleaned['Shank Width'].astype(float)

# Step 5: Rename columns
df_cleaned.rename(columns={
    'OMJ Style # ': 'SKUNo',
    'Elegant Jewelry Style #': 'StyleCoderough',
    'Quantity ': 'OrderQty',
    'SIZE': 'ItemSize',
    'PO # ': 'ItemPoNo'
}, inplace=True)

# Step 6: Add SrNo. as the first column
df_cleaned.insert(loc=0, column='SrNo.', value=range(1, len(df_cleaned) + 1))

# Step 7: Add empty 'StyleCode' column after 'SrNo.'
df_cleaned.insert(loc=1, column='StyleCode', value='')
df_cleaned.insert(
    loc=df_cleaned.columns.get_loc('ItemSize') + 1,
    column='ItemSizeCopy',
    value=df_cleaned['ItemSize'].map(lambda x: int(x) if x.is_integer() else round(x, 2))
)

# Step 8: Format ItemSize as string with 'UP' prefix
def format_item_size(x):
    if x == int(x):  # whole number like 7.0
        return f'UP{int(x):02d}'
    else:
        return f'UP{x}'

df_cleaned['ItemSize'] = df_cleaned['ItemSize'].map(format_item_size)

# Step 9: Reorder columns to have StyleCode, ItemSize, OrderQty together
cols = df_cleaned.columns.tolist()
# Move ItemSize after StyleCode
cols.insert(2, cols.pop(cols.index('ItemSize')))
# Move OrderQty after ItemSize
cols.insert(3, cols.pop(cols.index('OrderQty')))
df_cleaned = df_cleaned[cols]

orderqty_index = df_cleaned.columns.get_loc('OrderQty')
df_cleaned.insert(loc=orderqty_index + 1, column='OrderItemPcs', value=1)
df_cleaned.insert(loc=orderqty_index + 2, column='Metal', value='')
df_cleaned.insert(loc=orderqty_index + 3, column='Tone', value='')

# Step 11: Fill 'Tone' column with first letter of 'Metal Color '
df_cleaned['Tone'] = df_cleaned['Metal Color '].astype(str).str.strip().str[0]

# Step 12: Fill 'Metal' column as G + digits before 'K' in Metal Type + Tone (e.g. G14Y)
metal_karat = df_cleaned['Metal Type'].astype(str).str.extract(r'(?i)(\d+)(?=k)', expand=False).fillna('')
df_cleaned['Metal'] = 'G' + metal_karat + df_cleaned['Tone']

# Step 13: Move 'ItemPoNo' to the right side of 'Tone'
cols = df_cleaned.columns.tolist()
tone_index = cols.index('Tone')
itempono_index = cols.index('ItemPoNo')
itempono_col = cols.pop(itempono_index)
cols.insert(tone_index + 1, itempono_col)
df_cleaned = df_cleaned[cols]

itempono_index = df_cleaned.columns.get_loc('ItemPoNo')
df_cleaned.insert(loc=itempono_index + 1, column='ItemRefNo', value='')
df_cleaned.insert(loc=itempono_index + 2, column='StockType', value='')
df_cleaned.insert(loc=itempono_index + 3, column='MakeType', value='')
df_cleaned.insert(loc=itempono_index + 4, column='CustomerProductionInstruction', value='')
df_cleaned.insert(loc=itempono_index + 5, column='SpecialRemarks', value='')
df_cleaned.insert(loc=itempono_index + 6, column='DesignProductionInstruction', value='')
df_cleaned.insert(loc=itempono_index + 7, column='StampInstruction', value='')
df_cleaned.insert(loc=itempono_index + 8, column='OrderGroup', value='')
df_cleaned.insert(loc=itempono_index + 9, column='Certificate', value='')

sku_index = df_cleaned.columns.get_loc('SKUNo')
new_columns = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]
for i, col in enumerate(new_columns):
    df_cleaned.insert(loc=sku_index + 1 + i, column=col, value='')

# Step 10: Display result
df_cleaned.head(24)


,SrNo.,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,Expecteddeliverydate,SetPrice,StoneQuality,StyleCoderough,Metal Type,Metal Color,ItemSizeCopy,Shank Thickness,Shank Width,Diamod Quality
0,1,,UP9.5,1,1,G14Y,Y,ALGUL,,,...,,,,YR4171G,14K,YELLOW,9.50,1.51,2.80,W5A / G+- SI1
1,2,,UP8.25,1,1,G14W,W,001-02398,,,...,,,,YR4171G,14K,WHITE,8.25,1.51,2.80,W5A / G+- SI1
2,3,,UP07,1,1,G14Y,Y,001-00572,,,...,,,,YR4171F,14k,YELLOW,7.00,1.36,2.44,W5A / G+- SI1
4,4,,UP05,1,1,G14Y,Y,001-114604,,,...,,,,YR4171F,14K,YELLOW,5.00,1.36,2.44,W5A / G+- SI1
6,5,,UP09,2,1,G14W,W,ATHMANN,,,...,,,,YR4171SB,14K,WHITE,9.00,1.70,3.54,NaN
7,6,,UP06,1,1,G14W,W,ZAVAL,,,...,,,,YR4171S,14K,WHITE,6.00,1.63,3.41,NaN
9,7,,UP4.25,1,1,G14W,W,001-114604,,,...,,,,YR4321CMC,14K,WHITE,4.25,1.34,1.94,W5A / G+- SI1
10,8,,UP06,2,1,G14W,W,661070S,,,...,,,,RG0000361D,14K,WHITE,6.00,1.26,1.72,W5A / G+- SI1


In [21]:
#
import pandas as pd
import re

# Step 1: Read the actual data starting from row 11 (skip first 10 rows)
df = pd.read_excel(r'C:\Users\Admin\Desktop\OMJ_PO.xlsx', skiprows=10)

# Step 2: Select required columns
selected_columns = ['OMJ Style # ', 'Elegant Jewelry Style #', 'Quantity ', 
                    'Metal Type', 'Metal Color ', 'SIZE', 
                    'Shank Thickness', 'Shank Width', 'PO # ', 'Diamod Quality ']
df_selected = df[selected_columns]

# Step 3: Drop rows where 'OMJ Style # ' is missing
df_cleaned = df_selected.dropna(subset=['OMJ Style # ']).copy()

# Step 4: Convert numeric columns safely
df_cleaned['Quantity '] = pd.to_numeric(df_cleaned['Quantity '], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Quantity '].notna()]
df_cleaned['Quantity '] = df_cleaned['Quantity '].astype(int)

df_cleaned['SIZE'] = pd.to_numeric(df_cleaned['SIZE'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['SIZE'].notna()]
df_cleaned['SIZE'] = df_cleaned['SIZE'].astype(float)

df_cleaned['Shank Thickness'] = pd.to_numeric(df_cleaned['Shank Thickness'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Thickness'].notna()]
df_cleaned['Shank Thickness'] = df_cleaned['Shank Thickness'].astype(float)

df_cleaned['Shank Width'] = pd.to_numeric(df_cleaned['Shank Width'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Width'].notna()]
df_cleaned['Shank Width'] = df_cleaned['Shank Width'].astype(float)

# Step 5: Rename columns
df_cleaned.rename(columns={
    'OMJ Style # ': 'SKUNo',
    'Elegant Jewelry Style #': 'StyleCoderough',
    'Quantity ': 'OrderQty',
    'SIZE': 'ItemSize',
    'PO # ': 'ItemPoNo'
}, inplace=True)

# Step 6: Add SrNo. as the first column
df_cleaned.insert(loc=0, column='SrNo.', value=range(1, len(df_cleaned) + 1))

# Step 7: Add StyleCode column
df_cleaned.insert(loc=1, column='StyleCode', value='')

# Step 8: Create ItemSizeCopy with int if whole number else float rounded to 2 decimals
def convert_item_size_copy(x):
    return int(x) if float(x).is_integer() else round(float(x), 2)

df_cleaned.insert(
    loc=df_cleaned.columns.get_loc('ItemSize') + 1,
    column='ItemSizeCopy',
    value=df_cleaned['ItemSize'].map(convert_item_size_copy)
)

# Step 9: Format ItemSize as string with 'UP' prefix and leading zero if whole number
def format_item_size(x):
    if float(x).is_integer():
        return f'UP{int(float(x)):02d}'
    else:
        return f'UP{float(x)}'

df_cleaned['ItemSize'] = df_cleaned['ItemSize'].map(format_item_size)

# Step 10: Reorder columns: StyleCode, ItemSize, ItemSizeCopy, OrderQty
cols = df_cleaned.columns.tolist()
cols.insert(2, cols.pop(cols.index('ItemSize')))
cols.insert(3, cols.pop(cols.index('ItemSizeCopy')))
cols.insert(4, cols.pop(cols.index('OrderQty')))
df_cleaned = df_cleaned[cols]

# Step 11: Add OrderItemPcs, Metal, Tone columns
orderqty_index = df_cleaned.columns.get_loc('OrderQty')
df_cleaned.insert(loc=orderqty_index + 1, column='OrderItemPcs', value=1)
df_cleaned.insert(loc=orderqty_index + 2, column='Metal', value='')
df_cleaned.insert(loc=orderqty_index + 3, column='Tone', value='')

# Step 12: Fill 'Tone' column with first letter of 'Metal Color '
df_cleaned['Tone'] = df_cleaned['Metal Color '].astype(str).str.strip().str[0]

# Step 13: Fill 'Metal' as 'G' + digits before 'K' in Metal Type + Tone
metal_karat = df_cleaned['Metal Type'].astype(str).str.extract(r'(?i)(\d+)(?=k)', expand=False).fillna('')
df_cleaned['Metal'] = 'G' + metal_karat + df_cleaned['Tone']

# Step 14: Move 'ItemPoNo' column to right after 'Tone'
cols = df_cleaned.columns.tolist()
tone_index = cols.index('Tone')
itempono_col = cols.pop(cols.index('ItemPoNo'))
cols.insert(tone_index + 1, itempono_col)
df_cleaned = df_cleaned[cols]

# Step 15: Add additional columns after ItemPoNo
itempono_index = df_cleaned.columns.get_loc('ItemPoNo')
additional_after_pono = [
    'ItemRefNo', 'StockType', 'MakeType', 'CustomerProductionInstruction',
    'SpecialRemarks', 'DesignProductionInstruction', 'StampInstruction',
    'OrderGroup', 'Certificate'
]
for i, col in enumerate(additional_after_pono):
    df_cleaned.insert(loc=itempono_index + 1 + i, column=col, value='')

# Step 16: Add columns after SKUNo
sku_index = df_cleaned.columns.get_loc('SKUNo')
additional_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]
for i, col in enumerate(additional_after_sku):
    df_cleaned.insert(loc=sku_index + 1 + i, column=col, value='')

# Optional: When exporting to Excel, you can preserve formatting like this:


# Format ItemSizeCopy for CSV output (remove trailing .00)
def format_no_trailing_zeros(x):
    if float(x).is_integer():
        return str(int(x))
    else:
        return str(x)

df_cleaned['ItemSizeCopy'] = df_cleaned['ItemSizeCopy'].map(format_no_trailing_zeros)
df_cleaned['StyleCode'] = df_cleaned['StyleCoderough'].astype(str).str.strip() + '-' + df_cleaned['ItemSizeCopy']+ df_cleaned['Tone'] + 'G'
df_cleaned.drop(columns=['StyleCoderough'], inplace=True)

# ✅ Step 17: Fill 'CustomerProductionInstruction' with dynamic text
def generate_instruction(thickness, width):
    return (
        "PLEASE MAKE SURE THERE IS NO POROSITY , PLEASE MAKE SURE THERE IS NO BLACK NO MILKY STONES  "
        "NO COLOR TING IN THE DIAMONDS, THE DIAMOND SHANK THICKNESS "
        f"{round(thickness, 2)} & {round(width, 2)} WIDTH WITH +/-0.1MM TOLERANCE"
    )

df_cleaned['CustomerProductionInstruction'] = df_cleaned.apply(
    lambda row: generate_instruction(row['Shank Thickness'], row['Shank Width']),
    axis=1
)
# ✅ Step 18: Fill 'SpecialRemarks' with formatted string using required columns
def generate_special_remarks(sku, metal_type, metal_color, size_copy, diamond_quality):
    sku = str(sku).strip()
    metal_type = str(metal_type).strip().upper()
    metal_color = str(metal_color).strip().upper()
    size_copy = str(size_copy).strip()
    
    # If diamond quality is missing, leave it blank after "DIA QLTY"
    if pd.isna(diamond_quality) or str(diamond_quality).strip().upper() in ['NAN', '']:
        diamond_quality_str = ''
    else:
        diamond_quality_str = str(diamond_quality).strip().upper()
    
    return f"{sku}, {metal_type}, {metal_color}, Size {size_copy}, DIA QLTY {diamond_quality_str}"

df_cleaned['SpecialRemarks'] = df_cleaned.apply(
    lambda row: generate_special_remarks(
        row['SKUNo'],
        row['Metal Type'],
        row['Metal Color '],
        row['ItemSizeCopy'],
        row['Diamod Quality ']
    ),
    axis=1
)

# ✅ Step 19: Fill 'StampInstruction' column
def generate_stamp_instruction(metal_type, size_copy, sku):
    metal_type = str(metal_type).strip().upper()
    size_copy = str(size_copy).strip()
    sku = str(sku).strip().upper()
    return f"OMJ LOGO, {metal_type}, {size_copy}, {sku}, E"

df_cleaned['StampInstruction'] = df_cleaned.apply(
    lambda row: generate_stamp_instruction(
        row['Metal Type'],
        row['ItemSizeCopy'],
        row['SKUNo']
    ),
    axis=1
)

#df_cleaned.drop(columns=['ItemSizeCopy','Metal Type','Metal Color ','Shank Thickness','Shank Width'], inplace=True)  # uncomment this 

# Save to CSV
#df_cleaned.to_csv(r'C:\Users\Admin\Desktop\OMJ_PO_Cleaned.csv', index=False)    # uncomment this 


df_cleaned.head(24)


,SrNo.,StyleCode,ItemSize,ItemSizeCopy,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,...,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality,Metal Type,Metal Color,Shank Thickness,Shank Width,Diamod Quality
0,1,YR4171G-9.5YG,UP9.5,9.5,1,1,G14Y,Y,ALGUL,,...,,,,,,14K,YELLOW,1.51,2.80,W5A / G+- SI1
1,2,YR4171G-8.25WG,UP8.25,8.25,1,1,G14W,W,001-02398,,...,,,,,,14K,WHITE,1.51,2.80,W5A / G+- SI1
2,3,YR4171F-7YG,UP07,7,1,1,G14Y,Y,001-00572,,...,,,,,,14k,YELLOW,1.36,2.44,W5A / G+- SI1
4,4,YR4171F-5YG,UP05,5,1,1,G14Y,Y,001-114604,,...,,,,,,14K,YELLOW,1.36,2.44,W5A / G+- SI1
6,5,YR4171SB-9WG,UP09,9,2,1,G14W,W,ATHMANN,,...,,,,,,14K,WHITE,1.70,3.54,NaN
7,6,YR4171S-6WG,UP06,6,1,1,G14W,W,ZAVAL,,...,,,,,,14K,WHITE,1.63,3.41,NaN
9,7,YR4321CMC-4.25WG,UP4.25,4.25,1,1,G14W,W,001-114604,,...,,,,,,14K,WHITE,1.34,1.94,W5A / G+- SI1
10,8,RG0000361D-6WG,UP06,6,2,1,G14W,W,661070S,,...,,,,,,14K,WHITE,1.26,1.72,W5A / G+- SI1


In [7]:
columns_list = df_cleaned.columns.tolist()
print(columns_list)


['SrNo.', 'StyleCode', 'ItemSize', 'ItemSizeCopy', 'OrderQty', 'OrderItemPcs', 'Metal', 'Tone', 'ItemPoNo', 'ItemRefNo', 'StockType', 'MakeType', 'CustomerProductionInstruction', 'SpecialRemarks', 'DesignProductionInstruction', 'StampInstruction', 'OrderGroup', 'Certificate', 'SKUNo', 'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt', 'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality', 'Metal Type', 'Metal Color ', 'Shank Thickness', 'Shank Width', 'Diamod Quality ']


In [5]:
#
import pandas as pd
import re

# Step 1: Read the actual data starting from row 11 (skip first 10 rows)
df = pd.read_excel(r'C:\Users\Admin\Downloads\OMJ_FINISHED_PO.xlsx', skiprows=10)

# Step 2: Select required columns
selected_columns = ['OMJ Style # ', 'Elegant Jewelry Style #', 'Quantity ', 
                    'Metal Type', 'Metal Color ', 'SIZE', 
                    'Shank Thickness', 'Shank Width', 'PO # ', 'Diamod Quality ']
df_selected = df[selected_columns]

# Step 3: Drop rows where 'OMJ Style # ' is missing
df_cleaned = df_selected.dropna(subset=['OMJ Style # ']).copy()

# Step 4: Convert numeric columns safely
df_cleaned['Quantity '] = pd.to_numeric(df_cleaned['Quantity '], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Quantity '].notna()]
df_cleaned['Quantity '] = df_cleaned['Quantity '].astype(int)

df_cleaned['SIZE'] = pd.to_numeric(df_cleaned['SIZE'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['SIZE'].notna()]
df_cleaned['SIZE'] = df_cleaned['SIZE'].astype(float)

df_cleaned['Shank Thickness'] = pd.to_numeric(df_cleaned['Shank Thickness'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Thickness'].notna()]
df_cleaned['Shank Thickness'] = df_cleaned['Shank Thickness'].astype(float)

df_cleaned['Shank Width'] = pd.to_numeric(df_cleaned['Shank Width'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Width'].notna()]
df_cleaned['Shank Width'] = df_cleaned['Shank Width'].astype(float)

# Step 5: Rename columns
df_cleaned.rename(columns={
    'OMJ Style # ': 'SKUNo',
    'Elegant Jewelry Style #': 'StyleCoderough',
    'Quantity ': 'OrderQty',
    'SIZE': 'ItemSize',
    'PO # ': 'ItemPoNo'
}, inplace=True)

# Step 6: Add SrNo. as the first column
df_cleaned.insert(loc=0, column='SrNo.', value=range(1, len(df_cleaned) + 1))

# Step 7: Add StyleCode column
df_cleaned.insert(loc=1, column='StyleCode', value='')

# Step 8: Create ItemSizeCopy with int if whole number else float rounded to 2 decimals
def convert_item_size_copy(x):
    return int(x) if float(x).is_integer() else round(float(x), 2)

df_cleaned.insert(
    loc=df_cleaned.columns.get_loc('ItemSize') + 1,
    column='ItemSizeCopy',
    value=df_cleaned['ItemSize'].map(convert_item_size_copy)
)

# Step 9: Format ItemSize as string with 'UP' prefix and leading zero if whole number
def format_item_size(x):
    if float(x).is_integer():
        return f'UP{int(float(x)):02d}'
    else:
        return f'UP{float(x)}'

df_cleaned['ItemSize'] = df_cleaned['ItemSize'].map(format_item_size)

# Step 10: Reorder columns: StyleCode, ItemSize, ItemSizeCopy, OrderQty
cols = df_cleaned.columns.tolist()
cols.insert(2, cols.pop(cols.index('ItemSize')))
cols.insert(3, cols.pop(cols.index('ItemSizeCopy')))
cols.insert(4, cols.pop(cols.index('OrderQty')))
df_cleaned = df_cleaned[cols]

# Step 11: Add OrderItemPcs, Metal, Tone columns
orderqty_index = df_cleaned.columns.get_loc('OrderQty')
df_cleaned.insert(loc=orderqty_index + 1, column='OrderItemPcs', value=1)
df_cleaned.insert(loc=orderqty_index + 2, column='Metal', value='')
df_cleaned.insert(loc=orderqty_index + 3, column='Tone', value='')

# Step 12: Fill 'Tone' column with first letter of 'Metal Color '
df_cleaned['Tone'] = df_cleaned['Metal Color '].astype(str).str.strip().str[0]

# Step 13: Fill 'Metal' as 'G' + digits before 'K' in Metal Type + Tone
metal_karat = df_cleaned['Metal Type'].astype(str).str.extract(r'(?i)(\d+)(?=k)', expand=False).fillna('')
df_cleaned['Metal'] = 'G' + metal_karat + df_cleaned['Tone']

# Step 14: Move 'ItemPoNo' column to right after 'Tone'
cols = df_cleaned.columns.tolist()
tone_index = cols.index('Tone')
itempono_col = cols.pop(cols.index('ItemPoNo'))
cols.insert(tone_index + 1, itempono_col)
df_cleaned = df_cleaned[cols]

# Step 15: Add additional columns after ItemPoNo
itempono_index = df_cleaned.columns.get_loc('ItemPoNo')
additional_after_pono = [
    'ItemRefNo', 'StockType', 'MakeType', 'CustomerProductionInstruction',
    'SpecialRemarks', 'DesignProductionInstruction', 'StampInstruction',
    'OrderGroup', 'Certificate'
]
for i, col in enumerate(additional_after_pono):
    df_cleaned.insert(loc=itempono_index + 1 + i, column=col, value='')

# Step 16: Add columns after SKUNo
sku_index = df_cleaned.columns.get_loc('SKUNo')
additional_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]
for i, col in enumerate(additional_after_sku):
    df_cleaned.insert(loc=sku_index + 1 + i, column=col, value='')

# Optional: When exporting to Excel, you can preserve formatting like this:


# Format ItemSizeCopy for CSV output (remove trailing .00)
def format_no_trailing_zeros(x):
    if float(x).is_integer():
        return str(int(x))
    else:
        return str(x)

df_cleaned['ItemSizeCopy'] = df_cleaned['ItemSizeCopy'].map(format_no_trailing_zeros)
df_cleaned['StyleCode'] = df_cleaned['StyleCoderough'].astype(str).str.strip() + '-' + df_cleaned['ItemSizeCopy']+ df_cleaned['Tone'] + 'G'
df_cleaned.drop(columns=['StyleCoderough'], inplace=True)

# ✅ Step 17: Fill 'CustomerProductionInstruction' with dynamic text
def generate_instruction(thickness, width):
    return (
        "PLEASE MAKE SURE THERE IS NO POROSITY , PLEASE MAKE SURE THERE IS NO BLACK NO MILKY STONES  "
        "NO COLOR TING IN THE DIAMONDS, THE DIAMOND SHANK THICKNESS "
        f"{round(thickness, 2)} & {round(width, 2)} WIDTH WITH +/-0.1MM TOLERANCE"
    )

df_cleaned['CustomerProductionInstruction'] = df_cleaned.apply(
    lambda row: generate_instruction(row['Shank Thickness'], row['Shank Width']),
    axis=1
)
# ✅ Step 18: Fill 'SpecialRemarks' with formatted string using required columns
def generate_special_remarks(sku, metal_type, metal_color, size_copy, diamond_quality):
    sku = str(sku).strip()
    metal_type = str(metal_type).strip().upper()
    metal_color = str(metal_color).strip().upper()
    size_copy = str(size_copy).strip()
    
    # If diamond quality is missing, leave it blank after "DIA QLTY"
    if pd.isna(diamond_quality) or str(diamond_quality).strip().upper() in ['NAN', '']:
        diamond_quality_str = ''
    else:
        diamond_quality_str = str(diamond_quality).strip().upper()
    
    return f"{sku}, {metal_type}, {metal_color}, Size {size_copy}, DIA QLTY {diamond_quality_str}"

df_cleaned['SpecialRemarks'] = df_cleaned.apply(
    lambda row: generate_special_remarks(
        row['SKUNo'],
        row['Metal Type'],
        row['Metal Color '],
        row['ItemSizeCopy'],
        row['Diamod Quality ']
    ),
    axis=1
)

# ✅ Step 19: Fill 'StampInstruction' column
def generate_stamp_instruction(metal_type, size_copy, sku):
    metal_type = str(metal_type).strip().upper()
    size_copy = str(size_copy).strip()
    sku = str(sku).strip().upper()
    return f"OMJ LOGO, {metal_type}, {size_copy}, {sku}, E"

df_cleaned['StampInstruction'] = df_cleaned.apply(
    lambda row: generate_stamp_instruction(
        row['Metal Type'],
        row['ItemSizeCopy'],
        row['SKUNo']
    ),
    axis=1
)

#df_cleaned.drop(columns=['ItemSizeCopy','Metal Type','Metal Color ','Shank Thickness','Shank Width'], inplace=True)  # uncomment this 

# Save to CSV
#df_cleaned.to_csv(r'C:\Users\Admin\Desktop\OMJ_Finished_PO_Cleaned.csv', index=False)    # uncomment this 


df_cleaned.head(24)


,SrNo.,StyleCode,ItemSize,ItemSizeCopy,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,...,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality,Metal Type,Metal Color,Shank Thickness,Shank Width,Diamod Quality
0,1,YR4321FMC-6.5WG,UP6.5,6.5,1,1,G14W,W,2008,,...,,,,,,14K,WHITE,1.48,2.63,W5A / G+- SI1
1,2,YR4321FMC-6WG,UP06,6,1,1,G14W,W,2008,,...,,,,,,14K,WHITE,1.48,2.63,W5A / G+- SI1
2,3,YR4321FMC-5.5WG,UP5.5,5.5,1,1,G14W,W,2008,,...,,,,,,14K,WHITE,1.48,2.63,W5A / G+- SI1
3,4,YR4321FMC-4.5WG,UP4.5,4.5,1,1,G14W,W,2008,,...,,,,,,14K,WHITE,1.48,2.63,W5A / G+- SI1
4,5,YR4171X-4.25YG,UP4.25,4.25,1,1,G14Y,Y,JANVIER,,...,,,,,,14K,YELLOW,1.46,2.57,W5A / G+- SI1


In [19]:
import pandas as pd
import re

# Step 1: Read the actual data starting from row 11 (skip first 10 rows)
df = pd.read_excel(r'C:\Users\Admin\Downloads\OM_CASTING_PO.xlsx', skiprows=10)

# Step 2: Select required columns
selected_columns = ['OMJ Style # ', 'Elegant Jewelry Style #', 'Quantity ', 
                    'Metal Type', 'Metal Color ', 'SIZE', 
                    'Shank Thickness', 'Shank Width', 'PO # ']
df_selected = df[selected_columns]

# Step 3: Drop rows where 'OMJ Style # ' is missing
df_cleaned = df_selected.dropna(subset=['OMJ Style # ']).copy()

# Step 4: Convert numeric columns safely
df_cleaned['Quantity '] = pd.to_numeric(df_cleaned['Quantity '], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Quantity '].notna()]
df_cleaned['Quantity '] = df_cleaned['Quantity '].astype(int)

df_cleaned['SIZE'] = pd.to_numeric(df_cleaned['SIZE'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['SIZE'].notna()]
df_cleaned['SIZE'] = df_cleaned['SIZE'].astype(float)

df_cleaned['Shank Thickness'] = pd.to_numeric(df_cleaned['Shank Thickness'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Thickness'].notna()]
df_cleaned['Shank Thickness'] = df_cleaned['Shank Thickness'].astype(float)

df_cleaned['Shank Width'] = pd.to_numeric(df_cleaned['Shank Width'], errors='coerce')
df_cleaned = df_cleaned[df_cleaned['Shank Width'].notna()]
df_cleaned['Shank Width'] = df_cleaned['Shank Width'].astype(float)

# Step 5: Rename columns
df_cleaned.rename(columns={
    'OMJ Style # ': 'SKUNo',
    'Elegant Jewelry Style #': 'StyleCoderough',
    'Quantity ': 'OrderQty',
    'SIZE': 'ItemSize',
    'PO # ': 'ItemPoNo'
}, inplace=True)

# Step 6: Add SrNo. as the first column
df_cleaned.insert(loc=0, column='SrNo.', value=range(1, len(df_cleaned) + 1))

# Step 7: Add StyleCode column
df_cleaned.insert(loc=1, column='StyleCode', value='')

# Step 8: Create ItemSizeCopy with int if whole number else float rounded to 2 decimals
def convert_item_size_copy(x):
    return int(x) if float(x).is_integer() else round(float(x), 2)

df_cleaned.insert(
    loc=df_cleaned.columns.get_loc('ItemSize') + 1,
    column='ItemSizeCopy',
    value=df_cleaned['ItemSize'].map(convert_item_size_copy)
)

# Step 9: Format ItemSize as string with 'UP' prefix and leading zero if whole number
def format_item_size(x):
    if float(x).is_integer():
        return f'UP{int(float(x)):02d}'
    else:
        return f'UP{float(x)}'

df_cleaned['ItemSize'] = df_cleaned['ItemSize'].map(format_item_size)

# Step 10: Reorder columns: StyleCode, ItemSize, ItemSizeCopy, OrderQty
cols = df_cleaned.columns.tolist()
cols.insert(2, cols.pop(cols.index('ItemSize')))
cols.insert(3, cols.pop(cols.index('ItemSizeCopy')))
cols.insert(4, cols.pop(cols.index('OrderQty')))
df_cleaned = df_cleaned[cols]

# Step 11: Add OrderItemPcs, Metal, Tone columns
orderqty_index = df_cleaned.columns.get_loc('OrderQty')
df_cleaned.insert(loc=orderqty_index + 1, column='OrderItemPcs', value=1)
df_cleaned.insert(loc=orderqty_index + 2, column='Metal', value='')
df_cleaned.insert(loc=orderqty_index + 3, column='Tone', value='')

# Step 12: Fill 'Tone' column with first letter of 'Metal Color '
df_cleaned['Tone'] = df_cleaned['Metal Color '].astype(str).str.strip().str[0]

# Step 13: Fill 'Metal' as 'G' + digits before 'K' in Metal Type + Tone
metal_karat = df_cleaned['Metal Type'].astype(str).str.extract(r'(?i)(\d+)(?=k)', expand=False).fillna('')
df_cleaned['Metal'] = 'G' + metal_karat + df_cleaned['Tone']

# Step 14: Move 'ItemPoNo' column to right after 'Tone'
cols = df_cleaned.columns.tolist()
tone_index = cols.index('Tone')
itempono_col = cols.pop(cols.index('ItemPoNo'))
cols.insert(tone_index + 1, itempono_col)
df_cleaned = df_cleaned[cols]

# Step 15: Add additional columns after ItemPoNo
itempono_index = df_cleaned.columns.get_loc('ItemPoNo')
additional_after_pono = [
    'ItemRefNo', 'StockType', 'MakeType', 'CustomerProductionInstruction',
    'SpecialRemarks', 'DesignProductionInstruction', 'StampInstruction',
    'OrderGroup', 'Certificate'
]
for i, col in enumerate(additional_after_pono):
    df_cleaned.insert(loc=itempono_index + 1 + i, column=col, value='')

# Step 16: Add columns after SKUNo
sku_index = df_cleaned.columns.get_loc('SKUNo')
additional_after_sku = [
    'Basestoneminwt', 'Basestonemaxwt', 'Basemetalminwt', 'Basemetalmaxwt',
    'Productiondeliverydate', 'Expecteddeliverydate', 'SetPrice', 'StoneQuality'
]
for i, col in enumerate(additional_after_sku):
    df_cleaned.insert(loc=sku_index + 1 + i, column=col, value='')

# Format ItemSizeCopy for output (remove trailing .00)
def format_no_trailing_zeros(x):
    if float(x).is_integer():
        return str(int(x))
    else:
        return str(x)

df_cleaned['ItemSizeCopy'] = df_cleaned['ItemSizeCopy'].map(format_no_trailing_zeros)

# StyleCode: StyleCoderough-ItemSizeCopy+Tone+G
df_cleaned['StyleCode'] = df_cleaned['StyleCoderough'].astype(str).str.strip() + '-' + df_cleaned['ItemSizeCopy'] + df_cleaned['Tone'] + 'G'
df_cleaned.drop(columns=['StyleCoderough'], inplace=True)

# ✅ Step 17: Fill 'CustomerProductionInstruction'
def generate_instruction(thickness, width):
    return (
        "PLEASE MAKE SURE THERE IS NO POROSITY , PLEASE MAKE SURE THERE IS NO BLACK NO MILKY STONES  "
        "NO COLOR TING IN THE DIAMONDS, THE DIAMOND SHANK THICKNESS "
        f"{round(thickness, 2)} & {round(width, 2)} WIDTH WITH +/-0.1MM TOLERANCE"
    )

df_cleaned['CustomerProductionInstruction'] = df_cleaned.apply(
    lambda row: generate_instruction(row['Shank Thickness'], row['Shank Width']),
    axis=1
)

# ✅ Step 18: Fill 'SpecialRemarks' (only up to "DIA QLTY")
def generate_special_remarks(sku, metal_type, metal_color, size_copy):
    sku = str(sku).strip()
    metal_type = str(metal_type).strip().upper()
    metal_color = str(metal_color).strip().upper()
    size_copy = str(size_copy).strip()
    return f"{sku}, {metal_type}, {metal_color}, Size {size_copy}, DIA QLTY"

df_cleaned['SpecialRemarks'] = df_cleaned.apply(
    lambda row: generate_special_remarks(
        row['SKUNo'],
        row['Metal Type'],
        row['Metal Color '],
        row['ItemSizeCopy'],
        
    ),
    axis=1
)

# ✅ Step 19: Fill 'StampInstruction'
def generate_stamp_instruction(metal_type, size_copy, sku):
    metal_type = str(metal_type).strip().upper()
    size_copy = str(size_copy).strip()
    sku = str(sku).strip().upper()
    return f"OMJ LOGO, {metal_type}, {size_copy}, {sku}, E"

df_cleaned['StampInstruction'] = df_cleaned.apply(
    lambda row: generate_stamp_instruction(
        row['Metal Type'],
        row['ItemSizeCopy'],
        row['SKUNo']
    ),
    axis=1
)

# ✅ Optional: Drop unnecessary columns if exporting
df_cleaned.drop(columns=['ItemSizeCopy','Metal Type','Metal Color ','Shank Thickness','Shank Width'], inplace=True)

# ✅ Save to CSV (uncomment to use)
df_cleaned.to_csv(r'C:\Users\Admin\Desktop\OMJ_CASTING_PO_Cleaned.csv', index=False)

# ✅ View top rows
df_cleaned.head(24)


,SrNo.,StyleCode,ItemSize,OrderQty,OrderItemPcs,Metal,Tone,ItemPoNo,ItemRefNo,StockType,...,Certificate,SKUNo,Basestoneminwt,Basestonemaxwt,Basemetalminwt,Basemetalmaxwt,Productiondeliverydate,Expecteddeliverydate,SetPrice,StoneQuality
0,1,YR4172SA-6WG,UP06,1,1,G14W,W,2007.0,,,...,,BSP3110-125,,,,,,,,
1,2,YR4171SB-8.5WG,UP8.5,1,1,G14W,W,2006.0,,,...,,VAL3310-140,,,,,,,,
2,3,YR4172H-6.5WG,UP6.5,2,1,G14W,W,2005.0,,,...,,BSP2712-100,,,,,,,,
3,4,YR4172H-6WG,UP06,2,1,G14W,W,2005.0,,,...,,BSP2712-100,,,,,,,,
